In [ ]:
from wabs import *
x = WABS()
d = xx.get_driver
wait = WebDriverWait(dd, 5)

In [ ]:
#xx.select_chat('Emergency')
#xx.click("//span[@class='_28HTg']")
#dx = xx.msgrd(10)

### Selenium & Xpath general

In [ ]:
def update_dict_values(a, b):
    '''updating key & values using append then set procedure'''
    c = {}
    for k, v in a.items():
        if k in list(b):
            c[k] = list(set(a[k] + b[k]))
        else:
            c[k] = a[k]
    return {**b, **c}

def sidepane_scroll(d, initial = 1, scrolls=50):
    for i in range(1, scrolls):
        d.execute_script("document.getElementById('pane-side').scrollTop={}".format(initial))
        initial = initial + i + 1
    return initial

def msg_scoll_up(d):
    d.find_element(By.XPATH, "//div[@class='_5kRIK']").send_keys(Keys.HOME)
    tm.sleep(2)
    
def wtx(d, xpt, wt=1):
    try: return d.find_element(By.XPATH, xpt).text
    except: return None

def ww(d, xpt, sec=2):
    wait = WebDriverWait(d, sec)
    try: return wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
    except: return None
    
def wx(d, xpt, elements=None, wait=None, text=None, click=None):
    try:
        if elements is not None:
            return d.find_elements(By.XPATH, xpt)
        elif wait is not none: 
            y = ww(d, xpt, wait)   
        else: wait is None: 
            y = d.find_element(By.XPATH, xpt)
        if text is not None: return y.text
        elif click is not None: 
            y.click()
            return 1
        else: return y
    except: return None
        
def write_msg(d, xpt, text):
    ac = ActionChains(d)
    ac.move_to_element(ww(d,xpt)).click().perform()
    if chr(10) not in text:
        ac.send_keys(text).send_keys(Keys.RETURN).perform()
    else:
        xx = text.split(chr(10))
        for line in xx:
            ac.send_keys(line).perform()
            ac.key_down(Keys.SHIFT).key_down(Keys.ENTER).key_up(Keys.SHIFT).key_up(Keys.ENTER).perform()
        ac.send_keys(Keys.RETURN).perform()
    
def action(d, k, v):
    ac = ActionChains(d)
    if '_sendtxt_' in k: write_msg(d, v[0], v[1])
    elif 'key_to_element': ac.send_keys_to_element(ww(d,v[0]), v[1]).send_keys(Keys.RETURN).perform()
    elif '_clk_' in k: ac.click(ww(d, v[0])).perform()
    elif '_justmove_' in k: ac.move_to_element(ww(d, v[0])).perform()
    elif '_hoverclick_' in k: 
        ac.move_to_element(ww(d, v[0])).pause(.5).perform()
        ac.click(ww(v[1])).perform()
    elif '_moveclick_' in k: ac.move_to_element(ww(d, v[0])).pause(1).click().perform()
    else: return None
    


### XPATH Factory

In [119]:
contains = lambda attr, value: "contains(" + attr + ",'" + value + "')"
starts_with = lambda attr, value: "starts-with(normalize-space(" + attr + "),'" + value + "')"
ends_with = lambda attr, value: "ends-with(normalize-space(" + attr + "),'" + value + "')"

contains_no_attr = lambda value: "contains(.,'" + value + "')"
starts_with_no_attr = lambda value: "starts-with(normalize-space(),'" + value + "')"
ends_with_no_attr = lambda value: "ends-with(normalize-space(),'" + value + "')"

text_equal = lambda value : "contains(normalize-space(text())='" + value + "')"
not_contains = lambda attr, value: 'not(' + contains(attr, value) + ')'

def contains_tuple(func, n:tuple=('div','@class','_3wQ5i')):
    if type(n) is str: return None,func('.', n)
    elif len(n)==2:return None,func(n[0], n[1])
    elif len(n)==3:return n[0],func(n[1], n[2])
    else:
        print('function contains_mix_tuple(n:tuple): can only handdle 3 element as (tag, attr, value)')
        return None, n

def contains_attr(list_str_tuple=[('@class', '_21Ahp'),('text()','Net Status'), '11:13 pm'], tag='*'):
    ls = list_str_tuple if type(list_str_tuple) is list else [list_str_tuple]
    x = [contains_tuple(contains, n)[1] if type(n) is tuple else contains('.', n)  for n in ls]
    return "//" + tag + '[' + ' and '.join(x) + ']'

def contains_add(xpath:str, tuple_arg:tuple=('attr', 'attr_value'), with_function:object=contains):
    '''xpath : existing xpath, tuple_arg: conition that transform and add into exting contains
    call :: contains_add("//span[contains(@class,'YESTERDAY')]",('.','TODAY'), not_expr)
    return :: 
    '''
    if xpath.rfind('contains') == -1:
        mid1, mid2 = contains_tuple(tuple_arg)
        print(mid1, mid2)
        if mid1 is None: return xpath + ' | *['+ mid2 + ']'
        else: return xpath + ' | ' + mid1 + '[' + mid2 + ']'
    else:
        start = xpath[:xpath.rfind(']',xpath.rfind('contains'))]
        mid1, mid2 = contains_tuple(with_function, tuple_arg)
        end = xpath[xpath.rfind(']',xpath.rfind('contains')):]
        return start + ' and '+ mid2 + end

def xpath_item(tag='*', attr='@class', attr_value='xyz'):
    return tag + '[' + attr + "='" + attr_value + "']"

def xpath_bulid(lsxp=[('div','@id', 'value'), ('span','@class', 'value')]):
    lx = []
    for n in lsxp:
        t, a, v = n
        lx.append(xpath_normal(t, a, v))
    else: return '//' + '//'.join(lx)

def xptgen_1(xpath_func, tag='*', attr=['.'], value=['TEXT']):
    ''' call: xptgen_1(contains, 'tag', ['attr','sdf'], ['value','231rq'])
        return: "tag[contains(@attr,'value') and contains(@sdf,'231rq')]"
        xpath_func are [contains, starts_with, ends_with, text_equal]
    '''
    ls = lambda x: x if type(x) is list else [x]
    tag = tag if tag is not None else "*"
    attr, value = ls(attr), ls(value)
    lx = []
    for a, b in zip(attr, value):
        if a.find('.') != -1: lx.append(xpath_func(a, b))
        else: lx.append(xpath_func('@' + a, b))
    xpg = tag + "[" + ' and '.join(lx) + "]"
    return xpg

def contains_attr_tag(list_str_tuple=[('@class', '_21Ahp'),('text()','Net Status'), '11:13 pm']):
    '''
    call: contains_mix([('@class', '_21Ahp'),('text()','Net Status'), '11:13 pm'])
    return: "//*[contains(@class,'_21Ahp') and contains(text(),'Net Status') and contains(.,'11:13 pm')]"
    '''
    ls = list_str_tuple if type(list_str_tuple) is list else [list_str_tuple]
    lc1, lc2 = [],  []
    for n in ls:
        if type(n) is tuple:
            r1, r2 = contains_mix_tuple(n)
            lc1.append(r1)
            if r2 is not None: lc2.append(r2)
        elif type(n) is str: lc1.append(contains('.',n))
        else: print("argument: 'list_str_tuple' can be string or list'")
    else: return lc1, lc2



def build(lstext, own, base, axes=None):
    '''contains_ls = [('@class', '_21Ahp'),('text()','Net Status'), '11:13 pm']'''
    anc = 'ancestor::' + base if axes is None else 'ancestor::' + axes
    return "\\" + base + contains_attr(own, contains_ls, contains)



['(',
 "//div[@data-testid='cell-frame-container']",
 "//div[@class='_8nE1Y']",
 ')',
 '[last()]/span/div[1]']

In [118]:
xpath_chunk("(//div[@data-testid='cell-frame-container']//div[@class='_8nE1Y'])[last()]/span/div[1]//div[@class='_MX'][7]//ancestor::div[@data-testid='msg-container']")


['(',
 "//div[@data-testid='cell-frame-container']",
 "//div[@class='_8nE1Y']",
 ')',
 '[last()]/span/div[1]',
 "//div[@class='_MX']",
 '[7]',
 "//ancestor::div[@data-testid='msg-container']"]

In [116]:
pickmid()

('//div[', "@class='_21Ahp'", ']')

### sr

In [106]:
x=

['(',
 "//div[@data-testid='cell-frame-container']",
 "//div[@class='_8nE1Y']",
 ')',
 '[last()]/span/div[1]',
 "//div[@class='_MX']",
 '[7]',
 '(',
 "//ancestor::div[@data-testid='msg-container']",
 ')']

In [70]:
def pickmid(s:str, start_key:str, upto_key:str):
    n1 = s.find(start_key)
    n2 = s.find(upto_key, n1)
    return s[:n1], s[n1:n2], s[n2:]

In [71]:
pickmid("//div[@data-testid='cell-frame-container']",'@',']')

('//div[', "@data-testid='cell-frame-container'", ']')

In [57]:
q = "//div[@data-testid='cell-frame-container']"
if q.startswith("//") and q.endswith("]") and q.find('=') == q.rfind('='):
    n1 = q.find('@')
    n2 = q.find(']',n1)
    q1 = q[:n1-1]
    q2 = q[n1-1:n2+1]
    q3= q[n2+1:]
    print(q1,'-', q2,'-', q3)

//div - [@data-testid='cell-frame-container'] - 


In [48]:
qq = "@class='_8nE1Y'"
print("contains(" + qq.replace('=',',') + ')')

contains(@class,'_8nE1Y')


In [ ]:
    if n is str:
        nx = n.replace('()',"*").replace(')','$)$').split('$').replace('*').replace('()')
        print(nx)
    else:
        print('ls:', n)

In [158]:
#contains_add("(//div[@id='main']//div[@data-testid='msg-container'])", ('@class','_11JPr'), contains)

In [160]:
#build(['Update','8:44'],"//div[@class='_21Ahp']//span[@dir='ltr']","(//div[@id='main']//div[@data-testid='msg-container'])")

In [ ]:
msg_container = ""
msg_container_except_quote = "//div[@class='ItfyB _3nbHh']"
msg = "//div[@class='_21Ahp']//span[@dir='ltr']"
msg_quote_sender = "//div[@data-testid='quoted-message']/div/div/div/div[1]/span"
msg_quote_text = "//div[@data-testid='quoted-message']//div[@dir='ltr']"

In [ ]:
"(//div[contains(@id='main') and contains(@data-testid='msg-container')]"

In [ ]:
"//div[contains(@data-testid,'quoted-message')]"

In [ ]:
"//div[@data-testid='quoted-message']/div/div/div/div[contains(position(),'1')]/span"
"//div[@data-testid='quoted-message']/div/div/div/div[contains(position(),'1') and span]"

In [ ]:
alt_plus_k_search_textarea = ('div','@class','_3wQ5i')

In [ ]:
#output = div[contains(@class,'_27K43 _31p5Q') and  ]

In [ ]:
not_expr(starts_with,'.','text')

In [ ]:
xpath_spliter()

In [ ]:
print(xpath_spliter())

In [ ]:
xptgen_1(contains, 'tag', ['attr','sdf'], ['value','231rq'])

In [ ]:
for i in x5:
     xl.insert(0, xpath_items_split(i))
else: return xl

In [ ]:
a = [('@class', '_21Ahp'),'Net Status', '11:13 pm']
for i in a:
    if type(i) is tuple:
        print()
    else:
        print(contains(i, '.'))

### msg panel functions

In [ ]:
msg_base = "(//div[@id='main']//div[@data-testid='msg-container'])"
msgbs = "//div[@class='ItfyB _3nbHh']"
msgrow = "/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[2]/div[3]/div[6]"
msg_text = '_21Ahp'
msg_quote = '_1hl2r'
msg_main_sender = "//div[@class='_27K43 _31p5Q']/div[1]//span[@dir='auto']"
mss = "//div[@class='_27K43 _31p5Q']/div[1]//span[@dir='auto']"
msg_main_text = "//div[@class='_21Ahp']//span[@dir='ltr']"
msg_main_time = "//div[@data-testid='msg-meta']"
msg_main_contain_image = "//div[@data-testid='image-thumb']"
msg_quote_sender = "//div[@data-testid='quoted-message']/div/div/div/div[1]/span"
msg_quote_msisdn = "//div[@data-testid='quoted-message']//span[@class='WJuYU']"
msg_quote_text = "//div[@data-testid='quoted-message']//div[@dir='ltr']"
msg_container_axes = "//ancestor::div[@data-testid='msg-container']"
conversation_weekday_str = "//span[@class='_11JPr']"
weekday_by_text = lambda text : "//span[contains(@class,'_11JPr') and contains(.,'" + str(text) + "')]"

self_msg = "//span[@aria-label='You:']"
others = "//span[@aria-label]"

msg_by_sender = lambda name : "(//div[@class='_27K43 _31p5Q'])/div[1]/span[contains(@dir,'auto') and contains(., '" + name + "')]"
msg_by_text_time = lambda text, time : "//div[contains(@class,'_21Ahp') and contains(., '" + text + "') and contains(.,'" + str(time) + "')]"
msg_container_by_sender = lambda name : msg_by_sender(name) + "//ancestor::div[@data-testid='msg-container']"
msg_container_by_text_time = lambda text, time : msg_by_text_time(text, time) + "//ancestor::div[@data-testid='msg-container']"

def msginfo(d, n=None, base="(//div[@id='main']//div[@data-testid='msg-container'])"):
    bi = base + '[' + str(n) + ']' if n is not None else base
    print('msgbase is being collecting:', bi)
    if ww(d, bi) is None: return None
    else:
        x.move_click(bi + "//div[@role='button'][contains(.,'Read more')]")
        dttm, sender = xx.msg_sender_dttm(bi)
        dc = {'sender' : [sender],
              'text' : [wtx(d, bi + "//div[@class='_21Ahp']/span[1]/span")],
              'q_sender' : [wtx(d, bi + "//div[@class='_3pMOs yKTUI']//div[1]/span")],
              'q_text' : [wtx(d, bi + "//div[@class='_3pMOs yKTUI']//div[2]/span")],
              'datetime' : [dttm],
              'sender_visible': [d, wtx(bi + "//div[@class='_27K43 _31p5Q']/div[1]//span[@dir='auto']")],
              'time': [wtx(d, bi + "//div[@data-testid='msg-meta']/span")]
              }
        dc['img'] = 'image' if wtx(d, bi + "//div[@data-testid='image-thumb']") is not None else 'NoImage'
        return dc

def iloop(d, loop_end=20, loop_st=1, dc = defaultdict(list)):
    nc = 0
    for n in range(loop_st, loop_end):
        dic = msginfo(d, 'last()-' + str(n))
        if dic is not None:
            if len(dc) == 0 : dc = dic
            else: dc = update_key_value(dc, dic)
            nc = 0
        else: 
            if nc>2: x.msg_scoll_up()
            if nc>1: x.just_move()
            elif nc>3: return dc
            else: print('nc: ', nc)
            nc = nc + 1
    else: return dc

### SIDE-PANE

In [ ]:
pane_chat = "//div[@id='pane-side']"
pane_chat_base = "//div[@data-testid='cell-frame-container']"
pane_chat_text_base = "//div[@class='_8nE1Y']"
pane_chat_name = "//div[@class='y_sn4']//span"
pane_chat_last_time = "//div[@class='y_sn4']//div[2]"
pane_chat_last_msg = "//div[@class='vQ0w7']//span[@dir='ltr']"
pane_chat_group_last_msg_sender = "//div[@class='vQ0w7']//span[@dir='auto']"
pane_chat_unread_count = "//span[@data-testid='icon-unread-count']"
pane_chat_axes = "//ancestor::" + pane_chat_base.replace('//','')
pane_chat_by_name = lambda chatname: pane_chat_name + "[contains(.,'" + chatname + "')]//ancestor::" + wapane_chat_base.replace('//','')
wp_chat_name = lambda chatname : pane_chat_by_name(chatname)
wp_chat_last_msg = lambda chatname : pane_chat_by_name(chatname) + pane_chat_last_msg
wp_chat_last_msg_time = lambda chatname : pane_chat_by_name(chatname) + pane_chat_last_time
wp_chat_unread_count_byname = lambda chatname : pane_chat_by_name(chatname) + pane_chat_unread_count
wapane_chat_by_text = lambda text: "(" + pane_chat_base + pane_chat_text_base + ")[contains(.,'" + text + "')]"
wapane_chat_by_name_4select = lambda chatname: pane_chat_name + "[contains(.,'" + chatname + "')]"

def update_key_value(a, b):
    '''updating key & values (values will be appened for existing key)'''
    c = {}
    if len(a) == 0: return b
    tls = lambda ls : ls if type(ls) is list else [ls]
    for k, v in a.items():
        c[k] = tls(a[k]) + tls(b[k]) if k in list(b) else tls(a[k])
    else: return {**b, **c}
    
xpt_by_chatname = lambda chatname: "//div[@class='_21S-L']//span[@dir='auto' and contains(text(),'" + chatname + "')]//ancestor::div[@class='_8nE1Y']"

def wapane_chat_type(d, bs):
    xpt = bs + "//div[@data-testid='chatlist-status-v3-ring']"
    if ww(d, xpt.strip(' ')) is None: return 'group'
    else: return 'contact'
    
def side_pane(d, n=None, pane_base="//div[@id='pane-side']//div[@data-testid='cell-frame-container']"):
    base = '(' + pane_base + "//div[@class='_8nE1Y'])" if '_8nE1Y' not in pane_base else pane_base
    bi = base + '[' + str(n) + ']' if n is not None else base
    if ww(d, bi) is None: 
        return None
    else: print('! ', wtx(d,bi))
    dc = {'chat_type': [wapane_chat_type(d, '(' + pane_base + ')' + '[' + str(n) + ']')],
        'chat_name' : [bi + "//div[@class='y_sn4']//span"],
        'last_text' : [bi + "//div[@class='vQ0w7']//span[@dir='ltr']"],
        'last_sender' : [bi + "//div[@class='vQ0w7']//span[@dir='auto']"],
        'last_msg_time' : [bi + "/div[@class='y_sn4']//div[@class='Dvjym']"],
        'new_msg_notif' : [bi + "//span[@data-testid='icon-unread-count']"]}
    for k, v in dc.items():
        v = [v] if type(v) is str else v
        dc[k] = [wtx(d, j) for j in v]
    return dc


def is_duplicate(dmain, dnew, key_list=['chat_name', 'last_text']):
    try:
        duplicate = []
        for n in key_list:
            if dnew[n] in dmain[n]: duplicate.append(True)
            else: pass
        if len(key_list) == len(duplicate): return True
        else: return False
    except: return False
    

def iwhile(d, x, end=20, start=1, base=None, from_last=False, sdc=defaultdict()):
    n, nc, sc = 1, 0, 1
    while end > n:
        inx = n if from_last == False else 'last()' - str(n)
        dc = side_pane(d, n=inx, pane_base=base) if base is not None else side_pane(d, n=inx)
        print(dc)
        if dc is not None:
            if len(sdc) == 0: sdc = dc
            else:
                if is_duplicate(sdc, dc) == False:
                    sdc = update_dict_values(sdc, dc)
                    nc, n = 0, n+1
                else:
                    if nc>1: x.just_move()
                    nc = nc + 1
        else:
            if nc>2 and nc<3:
                sc = sidepane_scroll(d,sc)
                n = 1
            elif nc>4: return dc
            nc = nc + 1
    print('ret')
    return dc

In [ ]:
wapane_chat_type(d, "(//div[@id='pane-side']//div[@data-testid='cell-frame-container']")

In [ ]:
iwhile(dd, xx, 5009)

In [ ]:
sidepane_scroll(d)

In [ ]:
"/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[2]/div[3]/div[10]/div/div/div[2]/div[1]"
/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[2]/div[3]/div[10]/div/div/div[2]/div[1]/div[1]
/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[2]/div[3]/div[10]/div/div/div[2]/div[1]/div[2]/div[1]/div/div/div
/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[2]/div[3]/div[10]/div/div/div[2]/div[1]/div[2]/div[1]/div/div/div/div/div[1]
/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[2]/div[3]/div[10]/div/div/div[2]/div[1]/div[2]/div[1]/div/div/div/div/div[2]
/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[2]/div[3]/div[10]/div/div/div[2]/div[1]/div[2]/div[2]/span[1]/span
/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[2]/div[3]/div[10]/div/div/div[2]/div[1]/div[3]/div/span

In [ ]:
xx.xpbulk({'text': "/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[2]/div[3]/div[9]/div/div/div[2]/div[1]/div[2]/div[2]/span[1]/span",
'time': "/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[2]/div[3]/div[7]/div/div/div[2]/div[1]/div/div[5]/div/span",
'qs': "/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[2]/div[3]/div[7]/div/div/div[2]/div[1]/div/div[2]/div/div/div/div/div[2]",
'qt': "/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[2]/div[3]/div[7]/div/div/div[2]/div[1]/div/div[2]/div/div/div/div/div[1]",
'sn': "/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[2]/div[3]/div[7]/div/div/div[2]/div[1]/div/div[1]/div"})

In [ ]:
aq = {'te': "(//div[@id='main']//div[@data-testid='msg-container'])/div[4]/div/span[1]/span",
    'ti': "(//div[@id='main']//div[@data-testid='msg-container'])/div[5]/div/span",
    'qs': "(//div[@id='main']//div[@data-testid='msg-container'])/div[2]/div/div/div/div/div[2]",
    'qt': "(//div[@id='main']//div[@data-testid='msg-container'])/div[2]/div/div/div/div/div[1]",
    'sn': "(//div[@id='main']//div[@data-testid='msg-container'])/div[1]/div"}

In [ ]:
xx.action('Emergency SOC Group', 'High fiber loss pending update',forward='Operations')

In [ ]:
xx.action('Emergency SOC Group', 'Update of Site Down',forward='OPS & RO')

In [ ]:
xp = dd.find_element(By.XPATH, "(//div[@id='main']//div[@data-testid='msg-container'])[last()-2]").get_attribute('innerHTML')
tree = etree.fromstring(xp)

In [ ]:
from lxml.html import fromstring
soup = BeautifulSoup(xp, 'lxml')
dom = etree.HTML(str(soup))

In [ ]:
root = fromstring(xp)

In [ ]:
x  = root.xpath("//div[@class='_21Ahp']")
for i in x:
    print(i.get_text())

In [ ]:
[item.strip().get_text for item in root.xpath("(//div[@class='_21Ahp'])")

In [ ]:
for i in c:
    print(i.text)